In [20]:
import sys
import re
from typing import List, Tuple
from collections import Counter
import matplotlib.pyplot as plt

In [21]:
sys.path.append("../")

In [22]:
file_path = 'input_ag.txt'

In [23]:
robots_description = []
with open(file_path, 'r') as file:
    for line in file:
        robots_description.append(line.strip('\n'))

In [24]:
robots_description

['p=40,20 v=-97,-64',
 'p=68,91 v=-80,-44',
 'p=26,102 v=95,62',
 'p=40,30 v=2,-82',
 'p=68,96 v=-64,70',
 'p=16,22 v=9,-86',
 'p=20,50 v=-42,-62',
 'p=62,26 v=12,23',
 'p=27,69 v=47,23',
 'p=73,7 v=-83,32',
 'p=41,45 v=-57,-85',
 'p=41,97 v=17,-28',
 'p=66,101 v=-68,-50',
 'p=35,31 v=-29,-24',
 'p=75,33 v=39,30',
 'p=13,90 v=-88,71',
 'p=96,101 v=-88,-23',
 'p=10,46 v=96,33',
 'p=83,25 v=62,-73',
 'p=95,26 v=-35,30',
 'p=99,44 v=87,-5',
 'p=25,46 v=35,71',
 'p=33,83 v=-61,-27',
 'p=5,49 v=87,22',
 'p=47,31 v=-11,-7',
 'p=28,33 v=-61,-80',
 'p=75,43 v=82,-65',
 'p=52,54 v=6,2',
 'p=60,97 v=-70,65',
 'p=57,0 v=31,-50',
 'p=54,73 v=-97,-10',
 'p=16,45 v=-90,-87',
 'p=4,99 v=-52,62',
 'p=66,57 v=-24,-89',
 'p=40,84 v=2,81',
 'p=2,68 v=-75,74',
 'p=21,8 v=95,-77',
 'p=14,70 v=17,57',
 'p=98,23 v=85,27',
 'p=94,12 v=35,90',
 'p=20,67 v=-25,-8',
 'p=9,81 v=59,-16',
 'p=4,64 v=21,-27',
 'p=73,64 v=91,60',
 'p=19,71 v=13,90',
 'p=34,69 v=-63,-61',
 'p=34,73 v=42,92',
 'p=81,26 v=-20,25',
 'p=3

In [25]:
def parse_robot(line: str) -> Tuple[Tuple[int, int], Tuple[int, int]]:
    """
    Parses a line of input and returns position and velocity as tuples.
    """
    match = re.match(r"p=(-?\d+),(-?\d+) v=(-?\d+),(-?\d+)", line.strip())
    px, py, vx, vy = map(int, match.groups())
    return (px, py), (vx, vy)

def compute_new_position(p: Tuple[int, int], v: Tuple[int, int], 
                        time: int, width: int, height: int) -> Tuple[int, int]:
    """
    Computes the new position after a given time with wrapping.
    """
    new_x = (p[0] + v[0] * time) % width
    new_y = (p[1] + v[1] * time) % height
    return new_x, new_y

def determine_quadrant(x: int, y: int, 
                      width: int, height: int) -> int:
    """
    Determines the quadrant of a position.
    Returns quadrant number (1, 2, 3, 4) or 0 if on a midline.
    """  
    # Determine if on vertical midline
    if width % 2 == 1:
        mid_x = width // 2
        if x == mid_x:
            return 0
    else:
        mid_x1 = width // 2 - 1
        mid_x2 = width // 2
        if x == mid_x1 or x == mid_x2:
            return 0

    # Determine if on horizontal midline
    if height % 2 == 1:
        mid_y = height // 2
        if y == mid_y:
            return 0
    else:
        mid_y1 = height // 2 - 1
        mid_y2 = height // 2
        if y == mid_y1 or y == mid_y2:
            return 0

    # Determine quadrant
    if width % 2 == 1:
        if x < width // 2:
            horiz = 'left'
        else:
            horiz = 'right'
    else:
        if x < width // 2 - 1:
            horiz = 'left'
        else:
            horiz = 'right'
    
    if height % 2 == 1:
        if y < height // 2:
            vert = 'top'
        else:
            vert = 'bottom'
    else:
        if y < height // 2 - 1:
            vert = 'top'
        else:
            vert = 'bottom'
    
    if horiz == 'left' and vert == 'top':
        return 1
    elif horiz == 'right' and vert == 'top':
        return 2
    elif horiz == 'left' and vert == 'bottom':
        return 3
    elif horiz == 'right' and vert == 'bottom':
        return 4
    else:
        return 0

def calculate_safety_factor(input_lines: List[str], 
                            width: int = 101, height: int = 103, 
                            time: int = 100) -> int:
    """
    Calculates the safety factor after a given time.
    """
    robots = []
    for line in input_lines:
        pos, vel = parse_robot(line)
        robots.append((pos, vel))
    
    quadrant_counts = {1: 0, 2: 0, 3: 0, 4: 0}
    
    for pos, vel in robots:
        new_x, new_y = compute_new_position(pos, vel, time, width, height)
        quadrant = determine_quadrant(new_x, new_y, width, height)
        if quadrant in quadrant_counts:
            quadrant_counts[quadrant] += 1
    
    # Calculate safety factor
    safety_factor = 1
    for q in [1, 2, 3, 4]:
        safety_factor *= quadrant_counts[q]
    
    return safety_factor

In [26]:
calculate_safety_factor(
        robots_description, width=101, height=103, time=100
    )

222208000

## Second star

In [27]:
def parse_robot(line: str) -> Tuple[Tuple[int, int], Tuple[int, int]]:
    """
    Parses a line of input and returns position and velocity as tuples.
    Example input: "p=0,4 v=3,-3"
    """
    match = re.match(r"p=(-?\d+),(-?\d+) v=(-?\d+),(-?\d+)", line.strip())
    if not match:
        raise ValueError(f"Invalid line format: {line}")
    px, py, vx, vy = map(int, match.groups())
    return (px, py), (vx, vy)

def compute_new_position(p: Tuple[int, int], v: Tuple[int, int], 
                        time: int, width: int, height: int) -> Tuple[int, int]:
    """
    Computes the new position after a given time with wrapping.
    """
    new_x = (p[0] + v[0] * time) % width
    new_y = (p[1] + v[1] * time) % height
    return new_x, new_y

def display_positions(positions: List[Tuple[int, int]], width: int, height: int) -> None:
    """
    Displays the robot positions on the grid.
    """
    grid = [['.' for _ in range(width)] for _ in range(height)]
    
    # Mark robot positions with '#'
    for x, y in positions:
        if 0 <= y < height and 0 <= x < width:
            grid[y][x] = '#'
        else:
            # This should not happen due to modulo operation, but just in case
            print(f"Warning: Robot position out of bounds: ({x}, {y})")
    
    # Print the grid row by row
    for row in grid:
        print(''.join(row))

def plot_positions(positions: List[Tuple[int, int]], width: int, height: int) -> None:
    """
    Plots the robot positions using matplotlib.
    """
    if not positions:
        print("No positions to plot.")
        return
    
    x_coords, y_coords = zip(*positions)
    
    plt.figure(figsize=(10, 10))
    plt.scatter(x_coords, y_coords, marker='s', color='green')
    plt.xlim(-1, width)
    plt.ylim(-1, height)
    plt.gca().invert_yaxis()  # To match grid display where y=0 is at the top
    plt.title("Robot Positions at Symmetrical Time")
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.grid(True)
    plt.show()

def find_earliest_symmetry_time(input_lines: List[str], 
                                width: int = 101, height: int = 103, 
                                max_time: int = 20000) -> Tuple[int, List[Tuple[int, int]]]:
    """
    Finds the earliest time when robot positions are symmetrical across the y-axis.
    """
    robots = []
    for line in input_lines:
        pos, vel = parse_robot(line)
        robots.append((pos, vel))
    
    for t in range(max_time + 1):
        # Compute new positions for all robots at time t
        new_positions = [compute_new_position(pos, vel, t, width, height) for pos, vel in robots]
        
        # Count occurrences of each position
        position_counts = Counter(new_positions)
        
        # Create mirrored positions
        mirrored_positions = [((width - 1 - x), y) for (x, y) in new_positions]
        mirrored_counts = Counter(mirrored_positions)
        
        # Check if original counts match mirrored counts
        if position_counts == mirrored_counts:
            return t, new_positions
    
    raise ValueError(f"No symmetrical time found within {max_time} seconds.")

def calculate_robot_positions(input_lines: List[str], 
                              width: int = 101, height: int = 103, 
                              time: int = 100) -> List[Tuple[int, int]]:
    """
    Calculates the positions of all robots after a given time.
    """
    robots = []
    for line in input_lines:
        pos, vel = parse_robot(line)
        robots.append((pos, vel))
    
    new_positions = []
    for pos, vel in robots:
        new_x, new_y = compute_new_position(pos, vel, time, width, height)
        new_positions.append((new_x, new_y))
    
    return new_positions

In [28]:
if __name__ == "__main__":
    
    example_width = 101
    example_height = 103
    example_max_time = 100000
    
    try:
        symmetry_time, symmetry_positions = find_earliest_symmetry_time(
            robots_description, width=example_width, height=example_height, max_time=example_max_time
        )
        print(f"Earliest Symmetrical Time: {symmetry_time} seconds")
        print("Robot Positions at Symmetrical Time:")
        display_positions(symmetry_positions, example_width, example_height)
        
        plot_positions(symmetry_positions, example_width, example_height)
    except ValueError as e:
        print(e)

No symmetrical time found within 100000 seconds.


In [29]:
example_positions = calculate_robot_positions(
        robots_description, width=101, height=103, time=300
    )

display_positions(
        example_positions, width=101, height=103
    )

............#........#....#....................................................................#.#...
......................................#..........................................#...................
...................................#.......................##.....#..#..............#..........#.....
.#........##...............................#....#..#..................##..............#..........#...
....................#.........#....................#................................#................
.......................##.............#............................#.................................
.................#.##......#...........#...............#............#.....................#...#......
...#....................................#...................................................#........
..............................................#...........................#.....#..#.................
...................................................................#..............